In [41]:

from pandas_datareader import data as pdr
import yfinance as yfm
from tqdm import tqdm
yf.pdr_override()

ticker_df_list =list(data['Ticker'].unique())
price_data={}
for ticker in ticker_df_list:
    df = yf.download(ticker, interval='1d', period='3y')
    price_data[ticker] = df
    df.to_csv(f'stockPrice/{ticker}.csv')
    break

NameError: name 'yf' is not defined

In [ ]:

import os
import pandas as pd
from tqdm import tqdm
# Set the path to the directory containing the CSV files
PATH = '/home/ssu36/tiger/Stock_Prediction/data/stockPrice'

# Get the list of CSV files in the directory
csv_files = [file for file in os.listdir(PATH) if file.endswith('.csv')]

# Initialize an empty dictionary to store the dataframes
dfs = {}

# Iterate over each CSV file
for file in tqdm(csv_files):
  # Read the CSV file into a dataframe
  df = pd.read_csv(os.path.join(PATH, file))
  ticker_name =file.split('.')[0]
  df["Ticker"] = ticker_name
  # Add the dataframe to the dictionary with the file name as the key
  if len(df) == 755:
    dfs[ticker_name] = df
data = pd.concat(dfs.values(), ignore_index=True)
# Print the dictionary of dataframes


In [ ]:
# import datasets
# data= datasets.load_dataset('sehyun66/STOCKPRICE','NASDAQ_3y',split='train')
# data = data.to_pandas()





In [ ]:

def preprocess_window(df,feature:list,window_size:int,predict_size:int,sliding_size:int):
  ticker_number =len(df['Ticker'].unique())
  total_day = len(df['Date'].unique())
  window_number =int(total_day - (window_size+predict_size)/sliding_size+1)
  feature_size = len(feature)

  sequences = list()
  for group in df.groupby('Ticker'):
    sequences.append(group[1][feature])
  sequences=np.array(sequences)

  ## min_max trading
  price , trade,sentiments =np.split(sequences,[4,5],axis=2)
  trade , min_max_trading =min_max(trade)
  price , min_max_list =min_max(price)
  combine =np.concatenate([price,trade,sentiments],axis =2)

  ## min_max another
  result_list= []
  for i in range(0,window_number):
    a, b, c= np.split(combine,[i,i+window_size+predict_size],axis=1)
    result_list.append(b)
  result_array = np.array(result_list)
  train ,vaild = np.split(result_array,[-1],axis=0)
  # train_x,train_y=np.split(train.reshape(total_day%(window_size+predict_size)*ticker_number,window_size+predict_size,feature_size),[window_size],axis=1)
  train_x,train_y=np.split(train.reshape((window_number-1)*ticker_number,window_size+predict_size,feature_size),[window_size],axis=1)
  vaild_x,vaild_y=np.split(vaild.reshape(1*ticker_number,window_size+predict_size,feature_size),[window_size],axis=1)
  print(train_x.shape,train_y.shape,vaild_x.shape,vaild_y.shape,len(min_max_list))
  return train_x,train_y,vaild_x,vaild_y,min_max_list


In [ ]:
import torch
import numpy as np
feature = [ 'Open', 'High','Low','Close', 'Volume']

sequences = list()
for group in data.groupby('Ticker'):
    sequences.append(group[1][feature])
sequences=np.array(sequences)

In [ ]:
price , trade,sentiments =np.split(sequences,[4,5],axis=2)
trade , min_max_trading =min_max(trade)
price , min_max_list =min_max(price)
combine =np.concatenate([price,trade,sentiments],axis =2)

In [ ]:
def inverse_min_max(sequences,min_max):
  original = []
  for index,sequence in enumerate(sequences):
    v_min =min_max[index][0]
    v_max =min_max[index][1]
    sequence_restored = (sequence) * (v_max - v_min) + v_min
    original.append(sequence_restored)
  return np.array(original)

In [ ]:

def make_split(sequences,WINDOW_NUMBER,PREDICT_SIZE,WINDOW_SIZE,FEATURE_SIZE,TICKER_NUMBER):
  original_list =[]
  for i in range(0,WINDOW_NUMBER):
    a, b, c= np.split(sequences,[i,i+WINDOW_SIZE+PREDICT_SIZE],axis=1)
    original_list.append(b)
  print(len(original_list))
  result_array = np.array(original_list)
  train ,vaild = np.split(result_array,[-1],axis=0)
  train_x,train_y=np.split(train.reshape((WINDOW_NUMBER-1)*TICKER_NUMBER,WINDOW_SIZE+PREDICT_SIZE,FEATURE_SIZE),[WINDOW_SIZE],axis=1)
  vaild_x,vaild_y=np.split(vaild.reshape(1*TICKER_NUMBER,WINDOW_SIZE+PREDICT_SIZE,FEATURE_SIZE),[WINDOW_SIZE],axis=1)
  return train_x,train_y,vaild_x,vaild_y

In [ ]:
train_x,train_y,vaild_x,vaild_y = make_split(sequences,window_number,predict_size,window_size,feature_size,ticker_number)

In [ ]:
train_x,train_y,vaild_x,vaild_y = make_split(combine,window_number,predict_size,window_size,feature_size,ticker_number)

In [1]:
from data_processing import Window_maker
import datasets
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
from tqdm import tqdm
data= datasets.load_dataset('sehyun66/STOCKPRICE','NASDAQ_3y',split='train'
                            )
data = data.to_pandas()
window_size= 90
predict_size= 15
test_size = 15
sliding_size=1
window_maked = Window_maker(window_size,test_size,sliding_size,predict_size,data)


/home/ssu36/anaconda3/envs/tiger/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
original, scaled ,min_max_list= window_maked.preprocess_window()
# price , trade,sentiments =np.split(sequences,[4,5],axis=2)
# trade , min_max_trading =min_max(trade)
# price , min_max_list =min_max(price)

전처리 완료..


In [4]:
test = np.swapaxes(original[0], 0, 1)

In [5]:
test.shape

(90, 1358496, 5)

In [14]:
def make_dataset(x_data,y_data,y_original,target,window_maker,train = True):
    class BaseDataset(Dataset):
      def __init__(self, x_data, y_data):
          x_data = np.swapaxes(x_data, 0, 1)
          y_data = np.swapaxes(y_data, 0, 1)
          label_data = np.swapaxes(y_original, 0, 1)
          self.train = train
          self.lable_data =torch.tensor(label_data[:,:,window_maker.feature.index(target)],dtype = torch.float32)
          self.x_data = torch.tensor(x_data,dtype =torch.float32)
          self.y_data = torch.tensor(y_data[:,:,window_maker.feature.index(target)],dtype = torch.float32)
          self.min_max_list = torch.tensor(window_maker.min_max_list,dtype = torch.float32)
      def __getitem__(self, index):
          return self.x_data[:,index], self.y_data[:,index] ,self.lable_data[:,index] ,self.min_max_list[self.get_minmax_index(index)],index ,self.get_minmax_index(index)
      def get_minmax_index(self,index):
          if self.train == True:
            index = index%(window_maked.ticker_number)
          else:
            index = index%(window_maker.ticker_number)
          return index
      def __len__(self):
          return self.x_data.shape[0]
    return BaseDataset(x_data,y_data)

In [15]:
  train_x,train_y,vaild_x,vaild_y = scaled
  trian_x_o,train_y_0,vaild_x_o,vaild_y_o = original

  train_data = make_dataset(train_x,train_y,train_y_0,'Close',window_maked,train=True)
  test_data =  make_dataset(vaild_x,vaild_y,vaild_y_o,'Close',window_maked,train=False)

In [21]:
 train_dataloader = DataLoader(train_data, batch_size=64, shuffle=True,drop_last=True)
 test_dataloader = DataLoader(test_data, batch_size=64, shuffle=True,drop_last=True)

In [17]:
for batch in train_dataloader:
  train_x,train_y,train_y_o ,min_max_list,index, min_max_index= batch
  print(train_x.shape,train_y.shape,train_y_o.shape,min_max_list.shape)
  break

torch.Size([64, 90, 5]) torch.Size([64, 15]) torch.Size([64, 15]) torch.Size([64, 2])


In [24]:
torch.transpose(train_x,0,1).shape

torch.Size([90, 64, 5])

In [18]:
def inverse_min_max(sequences, min_max):
  original = []
  for index, sequence in tqdm(enumerate(sequences)):
    v_min = min_max[index][0]
    v_max = min_max[index][1]
    sequence_restored = (sequence) * (v_max - v_min) + v_min
    original.append(sequence_restored)
    result = torch.stack(original)
  return result


In [9]:
min_max_list[0]

tensor([0.5900, 3.9500])

In [12]:
len(min_max_list)

64

In [13]:
index

tensor([ 667189,  424204,   31772, 1148769,  534079,   85506,  275788,  272712,
        1163889, 1308429,  235974, 1201019,  640015,  182198,  559812,  486165,
         625105,  894589, 1043157,  401873,  315855,  586384,  140125,  831902,
        1351761, 1323146,  392330, 1057887,  905280, 1117382,  423205,  436705,
         871084, 1193691,  608391,  697101, 1146275,   53598,   89673,  169000,
          75987,  792669,  780190,  492949,  313071,  387208,  398759,  974736,
         490803,  115744, 1081713,  274104, 1312472,  279437, 1276491,  428555,
         118211,  778506,  835455,  897822,  586475,  573951,  923040,  502826])

In [14]:
min_max_index

tensor([ 757, 1276, 1868, 1737,   79,   66,  244, 1440, 1905, 1197, 1014,  587,
        1351,  638,  180, 1293, 1393, 1741,  789,  305, 1863, 1120, 1285,  998,
        1809,  962, 1442,  567, 1752,  254,  277,  961, 1732, 1803, 1767,  765,
        1379,  198, 2097,  256, 1227,  213,  550, 1669, 1215,  592, 1463,  720,
        1659,  400,  897,  696,  968, 1757, 1299, 1355,  731, 1002,  279,  702,
        1211, 1503,  288,  866])

In [15]:
train_x.shape,train_y.shape,train_y_0.shape

(torch.Size([64, 5, 90]), torch.Size([64, 15]), (1358496, 15, 5))

In [16]:
len(min_max_list)

64

In [21]:
result = Window_maker.inverse_min_max(train_y,min_max_list)


In [22]:
result[2],train_y_o[2]

(tensor([110.1400, 110.3500, 109.2000, 108.4375, 106.8725, 106.6025, 104.5550,
         104.2000, 102.7450, 105.7625, 102.0575, 103.8850, 102.2200,  99.1650,
         100.3425]),
 tensor([110.1400, 110.3500, 109.2000, 108.4375, 106.8725, 106.6025, 104.5550,
         104.2000, 102.7450, 105.7625, 102.0575, 103.8850, 102.2200,  99.1650,
         100.3425]))

# PUSH CSV TO HUGGINGFACE 

In [ ]:
import datasets
import pandas as pd


dataset = datasets.load_dataset('sehyun66/STOCKPRICE','NASDAQ_3y')
data= pd.read_csv('NASDAQ_3y/stock_price_clean-train.csv',index_col=0)
data.to_csv('NASDAQ_3y/stock_price_clean-train.csv',index=False)
dataset.push_to_hub('sehyun66/STOCKPRICE','NASDAQ_3y')